In [1]:
import numpy as np

In [2]:
from collections import deque

In [3]:
import mediapipe as mp


In [4]:
import cv2

In [5]:
bpoints=[deque(maxlen=1024)]
rpoints=[deque(maxlen=1024)]
gpoints=[deque(maxlen=1024)]
ypoints=[deque(maxlen=1024)]

In [6]:
#this will tell which color is selected at a moment
red_index=0
blue_index=0
green_index=0
yellow_index=0

In [7]:
kernel=np.ones((5,5),np.uint8)

In [8]:
colors=[(255,0,0),(0,255,0),(0,0,255),(0,255,255)]

In [9]:
colorIndex=0

In [10]:
#for making the white background
paintwindow=np.zeros((471,636,3))+255
paintwindow=cv2.rectangle(paintwindow,(40,1),(140,65),(0,0,0),2)
paintwindow=cv2.rectangle(paintwindow,(160,1),(255,65),colors[0],2)
paintwindow=cv2.rectangle(paintwindow,(275,1),(370,65),colors[1],2)
paintwindow=cv2.rectangle(paintwindow,(390,1),(485,65),colors[2],2)
paintwindow=cv2.rectangle(paintwindow,(505,1),(600,65),colors[3],2)

In [11]:
cv2.putText(paintwindow,"CLEAR",(49,33),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)
cv2.putText(paintwindow,"BLUE",(185,33),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)
cv2.putText(paintwindow,"GREEN",(298,33),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)
cv2.putText(paintwindow,"RED",(420,33),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)
cv2.putText(paintwindow,"YELLOW",(520,33),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)
cv2.namedWindow("Paint",cv2.WINDOW_AUTOSIZE)

In [12]:
#initialize mediapipe
mpHands=mp.solutions.hands
hands=mpHands.Hands(max_num_hands=1,min_detection_confidence=0.7)
mpDraw=mp.solutions.drawing_utils #for making landmarks



In [14]:
#intialize web cam
cap=cv2.VideoCapture(0)
ret,frame_temp=cap.read()
while ret:
    ret,frame=cap.read()
    x,y,c=frame.shape
    #flip the frame vertically
    frame=cv2.flip(frame,1)
    framergb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

    frame=cv2.rectangle(frame,(40,1),(140,65),(0,0,0),2)
    frame=cv2.rectangle(frame,(160,1),(255,65),colors[0],2)
    frame=cv2.rectangle(frame,(275,1),(370,65),colors[1],2)
    frame=cv2.rectangle(frame,(390,1),(485,65),colors[2],2)
    frame=cv2.rectangle(frame,(505,1),(600,65),colors[3],2)
    cv2.putText(frame,"CLEAR",(49,33),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)
    cv2.putText(frame,"RED",(185,33),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)
    cv2.putText(frame,"GREEN",(298,33),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)
    cv2.putText(frame,"BLUE",(420,33),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)
    cv2.putText(frame,"YELLOW",(520,33),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),2,cv2.LINE_AA)

    result=hands.process(framergb)#get hand landmark prediction

    if result.multi_hand_landmarks:
        landmarks=[]
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                lx=int(lm.x*640)#adjust acc to frame size as their values lie between 0 and 1
                ly=int(lm.y*480)
                landmarks.append([lx,ly])
            mpDraw.draw_landmarks(frame,handslms,mpHands.HAND_CONNECTIONS)
        fore_finger=(landmarks[8][0],landmarks[8][1])
        center=fore_finger
        thumb=(landmarks[4][0],landmarks[4][1])
        cv2.circle(frame,center,3,(0,255,0),-1)
        
        if(thumb[1]-center[1]<30):
            bpoints.append(deque(maxlen=512))
            blue_index+=1
            gpoints.append(deque(maxlen=512))
            green_index+=1
            rpoints.append(deque(maxlen=512))
            red_index+=1
            ypoints.append(deque(maxlen=512))
            yellow_index+=1
        elif(center[1]<=65):
            if(40<=center[0]<=140):# if finger in the clear box
                bpoints=[deque(maxlen=512)]
                rpoints=[deque(maxlen=512)]
                gpoints=[deque(maxlen=512)]
                ypoints=[deque(maxlen=512)]

                red_index=0
                blue_index=0
                green_index=0
                yellow_index=0
                paintwindow[67:,:,:]=255
            elif(160<=center[0]<=255):
                colorIndex=0
            elif(275<=center[0]<=370):
                colorIndex=1
            elif(390<=center[0]<=485):
                colorIndex=2
            elif(505<=center[0]<=600):
                colorIndex=3


        else:
            if colorIndex==0:
                bpoints[blue_index].appendleft(center)
            if colorIndex==1:
                gpoints[green_index].appendleft(center)
            if colorIndex==2:
                rpoints[red_index].appendleft(center)    
            if colorIndex==3:
                ypoints[yellow_index].appendleft(center)
        #append the next deques when nothing is detected to avoid messing up
    else:
            bpoints.append(deque(maxlen=512))
            blue_index+=1
            gpoints.append(deque(maxlen=512))
            green_index+=1
            rpoints.append(deque(maxlen=512))
            red_index+=1
            ypoints.append(deque(maxlen=512))
            yellow_index+=1
    points=[bpoints,gpoints,rpoints,ypoints] 
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1,len(points[i][j])):
                if points[i][j][k-1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame,points[i][j][k-1],points[i][j][k],colors[i],2)    
                cv2.line(paintwindow,points[i][j][k-1],points[i][j][k],colors[i],2)
    cv2.imshow("Output",frame)
    cv2.imshow("Paint",paintwindow)

    if(cv2.waitKey(1)==ord('q')):
        break
cap.release()
cv2.destroyAllWindows()
 

C:\Users\harsh\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
